In [1]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
path_to_data_file = 'C:/Download/OneDrive/git_repo/projectcode/strava_vivacity_comparison/data/'
vivacity_folder = 'raw/vivacity'
strava_folder = 'raw/strava'

def get_csv_files(folder_name: str, path_to_data_file: str = path_to_data_file):
    # csv files in the path
    files = glob.glob(os.path.join(path_to_data_file, folder_name, '*.csv'))
    return [file for file in files if file.endswith('.csv')]

# Load Vivacity Data

In [3]:
# load vivacity data
vivacity_files = get_csv_files(folder_name=vivacity_folder)
vivacity_raw = pd.read_csv(*vivacity_files, parse_dates=['LocalTime'], dtype={'cyclist': np.int8})
# change data types
vivacity_raw

,LocalTime,countlineid,countlinename,cyclist,direction
0,2021-01-25 00:00:00,15998,S10 MosquitoWay,0,in
1,2021-01-25 00:00:00,15998,S10 MosquitoWay,0,out
2,2021-01-25 00:00:00,15997,S10 MosquitoWay path,0,in
3,2021-01-25 00:00:00,15997,S10 MosquitoWay path,0,out
4,2021-01-25 00:00:00,16166,S11 MosquiteWay road LHS,0,out
...,...,...,...,...,...
9907195,2021-04-25 07:40:00,16155,S11 MosquiteWay road RHS,0,in
9907196,2021-04-25 07:40:00,16154,S12 MosquitoWay pedcrossing,0,in
9907197,2021-04-25 07:40:00,16154,S12 MosquitoWay pedcrossing,0,out
9907198,2021-04-25 07:40:00,16156,S13 MosquitoWay path,0,in


In [4]:
def clean_vivacity_data(df):
    """Function that group bys the counline and resamples the Vivaicty data into hourly buckets."""
    
    df =  df.groupby(
        ['countlineid', 'countlinename']
    ).apply(lambda s: s.resample('H', on='LocalTime').sum()).drop(columns=['countlineid']).reset_index()
    df.cyclist = df.cyclist.astype(np.int8)
    return df

In [5]:
# resample the data into hourly bins
vivacity = clean_vivacity_data(vivacity_raw)

In [6]:
vivacity.head()

,countlineid,countlinename,LocalTime,cyclist
0,15968,S3 MosquitoWay path,2020-11-16 00:00:00,0
1,15968,S3 MosquitoWay path,2020-11-16 01:00:00,0
2,15968,S3 MosquitoWay path,2020-11-16 02:00:00,0
3,15968,S3 MosquitoWay path,2020-11-16 03:00:00,0
4,15968,S3 MosquitoWay path,2020-11-16 04:00:00,0


# Load Strava data 

In [7]:
def merge_strava_data(strava_files):
    # not all columns are relevant for our task so only load relevant
    cols_to_load = ['edge_uid', 'hour', 'forward_trip_count', 'reverse_trip_count']
    dfs = [pd.read_csv(
        file,
        usecols=cols_to_load,
        parse_dates=['hour'],
        dtype={
            'forward_trip': np.int8,
            'reverse_trip_count': np.int8
        }
    ) for file in strava_files]
    return pd.concat(dfs, ignore_index=True)

In [8]:
# get strava file paths
strava_files = get_csv_files(folder_name=strava_folder)
strava = merge_strava_data(strava_files)
strava

,edge_uid,hour,forward_trip_count,reverse_trip_count
0,90327439,2021-04-20 18:00:00,0,5
1,90327439,2021-04-13 17:00:00,5,0
2,90327439,2021-04-22 17:00:00,0,5
3,90327439,2021-04-03 15:00:00,5,0
4,90327439,2021-04-13 18:00:00,5,0
...,...,...,...,...
9850587,297790525,2021-05-25 16:00:00,5,0
9850588,297790525,2021-05-22 14:00:00,0,5
9850589,297790525,2021-05-02 13:00:00,5,0
9850590,297790525,2021-05-09 11:00:00,5,5


In [9]:
# the sum of the strava data and drop columns
strava['strava_users'] = strava.iloc[:, 2] + strava.iloc[:, 3]
strava.drop(columns=['forward_trip_count', 'reverse_trip_count'], inplace=True)

# Save CSV into Processed Folder

In [10]:
# folder to save 
output_folder = 'processed/'
strava_output = os.path.join(path_to_data_file, output_folder, 'strava_processed.csv')
vivacity_output = os.path.join(path_to_data_file, output_folder, 'vivacity_processed.csv')

In [11]:
strava.to_csv(strava_output, index=False)
vivacity.to_csv(vivacity_output, index=False)

# Merge data

Combine the vivacity data and strava data based on the relationship between vivacity countline IDs and Strava edge IDs.

In [12]:
# ID table
relationship_table_path = os.path.join(path_to_data_file, "./raw/vivacity_StravaJoin_New.csv")

relationship_df = pd.read_csv(relationship_table_path)
relationship_df.head()

,countlineid,edge_id
0,23672,296373889
1,23676,296373129
2,23666,296384489
3,23716,296373488
4,16163,296374419


In [13]:
relationship_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   countlineid  58 non-null     int64
 1   edge_id      58 non-null     int64
dtypes: int64(2)
memory usage: 1.0 KB


In [14]:
# get all strava data that is are within the relationship tables
egde_ids = strava[strava.edge_uid.isin(relationship_df.edge_id.unique())]

# edge_uid == relationship_df.edge_id.unique())
egde_ids

,edge_uid,hour,strava_users
30778,296378039,2021-04-15 10:00:00,5
30779,296378039,2021-04-03 09:00:00,5
30780,296378039,2021-04-26 17:00:00,5
30781,296378039,2021-04-17 09:00:00,10
30782,296378039,2021-04-24 09:00:00,10
...,...,...,...
9655433,296384431,2021-05-01 10:00:00,5
9655434,296384431,2021-05-29 07:00:00,5
9842080,296383263,2021-05-29 09:00:00,5
9842081,296383263,2021-05-21 09:00:00,5


This gives us all the edge IDs that are within the Strava data. But lets find out which edge IDs are not within the Strava data. 

In [15]:
# check to see which edge_id relationships are not within the Strava data 
relationship_df[~relationship_df.edge_id.isin(strava.edge_uid.unique())]

,countlineid,edge_id
0,23672,296373889
1,23676,296373129
6,23722,296376276
14,16407,296383727
15,16154,296374391
18,23663,296375322
21,16157,296374331
26,23747,296376413
38,23665,296375318
39,16404,296376178


*written on 4th November 2021*

15 Edge IDs, therefore, 15 Vivcity Countlines are not within the Strava Data. 

In [16]:
# merge data
temp  = strava.merge(relationship_df, left_on='edge_uid', right_on='edge_id', how='inner')
combined = vivacity.merge(temp, left_on=['LocalTime', 'countlineid'], right_on=['hour', 'countlineid'], how='inner')
# temp # uncomment to see temp df
combined

,countlineid,countlinename,LocalTime,cyclist,edge_uid,hour,strava_users,edge_id
0,15968,S3 MosquitoWay path,2020-12-12 13:00:00,0,296383282,2020-12-12 13:00:00,5,296383282
1,15968,S3 MosquitoWay path,2020-12-15 08:00:00,1,296383282,2020-12-15 08:00:00,5,296383282
2,15968,S3 MosquitoWay path,2020-12-19 13:00:00,0,296383282,2020-12-19 13:00:00,5,296383282
3,15968,S3 MosquitoWay path,2020-12-20 09:00:00,3,296383282,2020-12-20 09:00:00,5,296383282
4,15968,S3 MosquitoWay path,2020-12-24 09:00:00,0,296383282,2020-12-24 09:00:00,5,296383282
...,...,...,...,...,...,...,...,...
2996,23749,S32 CometWay North outbound,2021-01-15 12:00:00,0,296376283,2021-01-15 12:00:00,5,296376283
2997,23749,S32 CometWay North outbound,2021-03-07 10:00:00,0,296376283,2021-03-07 10:00:00,5,296376283
2998,23749,S32 CometWay North outbound,2021-03-16 09:00:00,0,296376283,2021-03-16 09:00:00,5,296376283
2999,23749,S32 CometWay North outbound,2021-03-21 09:00:00,0,296376283,2021-03-21 09:00:00,5,296376283


In [17]:
# drop irrelevant rows 
combined.drop(columns=['edge_uid', 'hour'], inplace=True)

In [18]:
combined.head()

,countlineid,countlinename,LocalTime,cyclist,strava_users,edge_id
0,15968,S3 MosquitoWay path,2020-12-12 13:00:00,0,5,296383282
1,15968,S3 MosquitoWay path,2020-12-15 08:00:00,1,5,296383282
2,15968,S3 MosquitoWay path,2020-12-19 13:00:00,0,5,296383282
3,15968,S3 MosquitoWay path,2020-12-20 09:00:00,3,5,296383282
4,15968,S3 MosquitoWay path,2020-12-24 09:00:00,0,5,296383282


In [19]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3001 entries, 0 to 3000
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   countlineid    3001 non-null   int64         
 1   countlinename  3001 non-null   object        
 2   LocalTime      3001 non-null   datetime64[ns]
 3   cyclist        3001 non-null   int8          
 4   strava_users   3001 non-null   int64         
 5   edge_id        3001 non-null   int64         
dtypes: datetime64[ns](1), int64(3), int8(1), object(1)
memory usage: 143.6+ KB


In [20]:
combined[['cyclist', 'strava_users']].describe()

,cyclist,strava_users
count,3001.000000,3001.000000
mean,2.727091,6.012996
std,3.888937,2.721549
min,0.000000,5.000000
25%,0.000000,5.000000
50%,1.000000,5.000000
75%,4.000000,5.000000
max,25.000000,40.000000


In [21]:
combined[['cyclist', 'strava_users']].corr()

,cyclist,strava_users
cyclist,1.000000,0.010539
strava_users,0.010539,1.000000


In [22]:
combined['cyclist'].apply(lambda x: 5 if x<5 else int(5 * np.ceil(float(x) / 5))).corr(combined['strava_users'])

0.05546337827872942

In [23]:
from sklearn.metrics import r2_score

In [24]:
r2_score(y_pred=combined['cyclist'], y_true=combined['strava_users'])

-3.469972796950379

In [25]:
# check r2 score when rounding the data
r2_score(y_pred=combined['cyclist'].apply(lambda x: 5 if x<5 else int(5 * np.ceil(float(x) / 5))), y_true=combined['strava_users'])

-1.1309158010128413

In [26]:
r2_score(y_true=combined['cyclist'], y_pred=combined['strava_users'].apply(lambda x: 0 if x<5 else x)) # int(5 * np.ceil(float(x) / 5)))

-1.1891501791930796

The rounding of the Strava data is causing problems and prevents a good coefficient of determination socre. 

Lets explore further within the next notebook. 

In [27]:
combined_output = os.path.join(path_to_data_file, output_folder, 'combined.csv')
combined.to_csv(combined_output, index=False)